Redes Neuronales para Lenguaje Natural, 2025

---
# Laboratorio 2

En este laboratorio construiremos un sistema de Question Answering (QA) utilizando el método de Retrieval-Augmented Generation (RAG), que implica el uso de un paso de recuperación de información y un paso de generación de respuesta con LLM.

**Entrega: 18/11**

**Se debe entregar un archivo zip que contenga:**
* Este notebook de Python (.ipynb) completo.
* Los documentos obtenidos y utilizados como fuentes de información según se explica en la parte 1 (opcionalmente se puede entregar un archivo CSV con los textos de cada documento).
* Archivo CSV con el conjunto de preguntas y respuestas como se explica en la parte 5.

**No olvidar mantener todas las salidas de cada región de código en el notebook!**

---



In [ ]:
#@title Instalar librerias
!pip install transformers
!pip install bitsandbytes
!pip install accelerate
!pip install sentence-transformers
!pip install evaluate
!pip install bert_score
!pip install wikipedia-api
!pip install google-genai

In [ ]:
#@title Estilo de salida de colab
from IPython.display import HTML, display
pre_run_cell_fn = lambda: display(HTML('''<style> pre {white-space: pre-wrap;}</style>'''))
get_ipython().events.register('pre_run_cell', pre_run_cell_fn)

## Parte 1: Procesamiento de los documentos

En esta parte, cada grupo deberá construir y procesar su conjunto de documentos. Esto consiste de los siguientes pasos:

* Elegir un tema dentro de un dominio específico sobre el que trabajar.
* Obtener al menos 5 documentos en español que contengan información sobre el tema elegido.
* Procesar cada documento para extraer el texto del formato original a un string en Python (por ejemplo, extraer el texto de un PDF).

El resultado de esta parte debe ser una lista cargada en memoria que contenga el texto (string) de cada uno de los documentos elegidos.

**Sugerencias:**
* Se recomienda utilizar artículos de wikipedia para simplificar la etapa de extracción del texto (ver la librería [wikipedia-api](https://github.com/martin-majlis/Wikipedia-API/)).
* Opcionalmente puede utilizar documentos PDF, páginas web u otros formatos. En estos casos se sugiere:
  * Utilizar la librería PyPDF2 para procesar documentos PDF.
  * Utilizar la librería LangChain para procesar páginas web, en particular la clase Html2TextTransformer, que convierte HTML a Markdown ([ejemplo de uso](https://python.langchain.com/v0.2/docs/integrations/document_transformers/html2text/)).
* Puede ser conveniente guardar el resultado del procesamiento de los documentos en un archivo CSV (donde cada fila corresponde al texto de un documento) para no tener que repetir este proceso cada vez que se ejecuta el notebook, y en su lugar cargar el archivo CSV.

In [ ]:
# Su código aquí


In [ ]:
# Ejemplo de uso de la libraría wikipedia-api
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('RNLN2025', 'es')

page = wiki_wiki.page('Alan Turing')
print(page.text)

Los textos resultantes deben estar almacenados en la variable `documents`:

In [ ]:
documents = [] # Lista de strings con el texto de cada documento

## Parte 2: Chunking

Una vez que se obtiene el texto de cada documento, se debe realizar la etapa de _chunking_. Esta etapa consiste en dividir cada texto en segmentos más chicos a los que llamamos _chunks_.

Realizar la etapa de _chunking_ de forma automática utilizando un método simple que permita obtener _chunks_ de un largo aproximado de 500 caracteres.

Puede probar con dividir a nivel de caracteres, palabras o incluso párrafos, teniendo en cuenta que el largo de cada _chunk_ no debería exceder demasiado los 500 caracteres.

**Sugerencias:**
* Puede utilizar los splitters disponibles en LangChain ([documentación](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/)) como RecursiveCharacterTextSplitter, aunque no es obligatorio y también es correcto hacer una implementación propia.
* Tener en cuenta que esta etapa es crucial en el resultado final. Cuanto más contextualizados queden los *chunks*, mejor será el rendimiento de la etapa de recuperación de información. Es conveniente minimizar la división de palabras (o párrafos) por la mitad.

In [ ]:
def chunk_text(text):
  # Su código aquí

  return chunks # Lista de strings con los chunks del texto

In [ ]:
chunks = []
for document in documents:
  chunks += chunk_text(document)

## Parte 3: Recuperación de información

En esta parte vamos a implementar el método de recuperación de información que nos permitirá obtener los _chunks_ más relevantes para la pregunta.

En primer lugar, cargamos el modelo Bi-Encoder que utilizaremos para generar los embeddings utilizando la librería sentence_transformers.

Se utiliza el modelo multilingüe [intfloat/multilingual-e5-large](https://huggingface.co/intfloat/multilingual-e5-large), fine-tuning del modelo `xlm-roberta-large` para la tarea de generación de sentence embeddings.

Se pueden explorar otros modelos Bi-Encoder, e incluso modelos Cross-Encoder o del tipo ColBERT. En HuggingFace se puede consultar el siguiente [leaderboard](https://huggingface.co/spaces/mteb/leaderboard) que compara varios modelos de este tipo en diferentes tareas.

In [ ]:
from sentence_transformers import SentenceTransformer

model_emb = SentenceTransformer("intfloat/multilingual-e5-large")

A continuación se debe generar las representaciones vectoriales para todos los _chunks_ ([ejemplo de uso](https://huggingface.co/intfloat/multilingual-e5-large#support-for-sentence-transformers)).

**Observación:** El modelo que estamos usando espera que los _chunks_ comiencen con el prefijo `passage: ` por lo que será necesario agregarlo al inicio de todos los _chunks_.

In [ ]:
# Su código aquí

Por último, se debe implementar el algoritmo de búsqueda de los embeddings más cercanos para un embedding dado.

**Sugerencias:**
* Utilizar la clase NearestNeighbors de sklearn ([documentación](https://scikit-learn.org/dev/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)).

In [ ]:
# Su código aquí

## Parte 4: Generación de respuestas

### Configuración de LLM

En esta parte, implementaremos un wrapper flexible que permite experimentar con diferentes modelos de lenguaje:

1. **Llama 3.1** (modelo abierto): Utilizaremos el modelo Meta-Llama-3.1-8B-Instruct a través de HuggingFace.
2. **Gemini 2.0 Flash** (modelo cerrado): Utilizaremos la API de Google Gemini.

Para **Llama 3.1**, es necesario:
- Crearse una cuenta de HuggingFace (https://huggingface.co/)
- Aceptar los términos para usar el modelo en HuggingFace: https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct
- Crear un token de HuggingFace con permiso de lectura: https://huggingface.co/settings/tokens

Para **Gemini**, es necesario:
- Obtener una API key de Google AI Studio: https://aistudio.google.com/app/apikey

In [ ]:
# Ejecutar para conectarse a HuggingFace (solo necesario para Llama 3.1)
from huggingface_hub import notebook_login

notebook_login()

### Definición del Wrapper de LLM

A continuación se define una clase abstracta que permite intercambiar fácilmente entre diferentes modelos de lenguaje.

In [ ]:
from abc import ABC, abstractmethod
from typing import Optional
import os
from getpass import getpass

class LLMWrapper(ABC):
    """Clase base abstracta para wrappers de modelos de lenguaje."""
    
    @abstractmethod
    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        pass
    
    @abstractmethod
    def get_model_name(self) -> str:
        pass


class LlamaWrapper(LLMWrapper):
    def __init__(self):
        from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
        import torch
        
        print("Inicializando Llama 3.1...")
        
        self.tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct"
        )
        
        # Configuración de cuantización a 4 bits (para mejorar eficiencia)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
        
        # Inicializar el modelo
        self.model = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3.1-8B-Instruct",
            quantization_config=bnb_config,
            device_map="auto",
        )
        
        print("Llama 3.1 inicializado correctamente")
    
    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        from transformers import GenerationConfig, pipeline
        
        # Configuración de temperatura
        generation_config = GenerationConfig(
            temperature=temperature if temperature > 0 else None,
            do_sample=temperature > 0
        )
        
        # Inicializar pipeline para generación de texto
        pipe = pipeline(
            "text-generation",
            model=self.model,
            config=generation_config,
            tokenizer=self.tokenizer,
            pad_token_id=self.tokenizer.eos_token_id
        )
        
        # Generar texto
        output = pipe(
            prompt,
            return_full_text=False,
            max_new_tokens=max_tokens
        )
        
        return output[0]['generated_text']
    
    def get_model_name(self) -> str:
        return "Llama-3.1-8B-Instruct"


class GeminiWrapper(LLMWrapper):
    def __init__(self, api_key: Optional[str] = None):
        from google import genai
        
        print("Inicializando Gemini 2.0 Flash...")
        
        # Obtener API key
        if api_key is None:
            api_key = os.environ.get("GEMINI_API_KEY")
        
        if api_key is None:
            print("No se encontró la API key de Gemini en las variables de entorno.")
            api_key = getpass("Por favor, ingrese su API key de Gemini: ")
            # Guardar en variables de entorno para esta sesión
            os.environ["GEMINI_API_KEY"] = api_key
        
        self.client = genai.Client(api_key=api_key)
        
        print("Gemini 2.0 Flash inicializado correctamente")
    
    def generate(self, prompt: str, temperature: float = 0.0, max_tokens: int = 500) -> str:
        from google.genai import types
        
        # Configurar parámetros de generación
        config = types.GenerateContentConfig(
            temperature=temperature if temperature > 0 else 0.0,
            max_output_tokens=max_tokens,
        )
        
        # Generar respuesta
        response = self.client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt,
            config=config
        )
        
        return response.text
    
    def get_model_name(self) -> str:
        return "Gemini-2.0-Flash"

### Instanciar modelos

Seleccionar qué modelo(s) se va a inicializar. Se puede inicializar ambos para facilitar la experimentación posterior.

In [ ]:
# Descomentar el(los) modelo(s) que se desea utilizar

llama_model = LlamaWrapper()

gemini_model = GeminiWrapper()

# Seleccionar el modelo activo para los experimentos
active_model = llama_model  # o gemini_model

print(f"\nModelo activo: {active_model.get_model_name()}")

### Función auxiliar para generación de respuestas

Esta función utiliza el modelo activo seleccionado anteriormente.

In [ ]:
def get_response(
    prompt: str,
    model: LLMWrapper = None,
    temp: float = 0.0,
    max_tok: int = 500
) -> str:
    if model is None:
        model = active_model
    
    return model.generate(prompt, temperature=temp, max_tokens=max_tok)

### Crear prompt y generar respuesta

Escribir la función `create_prompt(question, use_chat_template=True)` que dada una pregunta, genere la prompt que se utilizará para generar la respuesta. Tener en cuenta que se debe realizar la búsqueda semántica de los _chunks_ más cercanos a la pregunta utilizando lo implementado en la parte 3.

**Observación:** Al igual que para los _chunks_, el modelo Bi-Encoder espera que la pregunta comience con un prefijo especial: `query: ` por lo que será necesario agregarlo al inicio de la pregunta para generar el embedding.

**Sugerencias:**
* Puede probar con distintas cantidades de _chunks_ recuperados, pero se sugiere comenzar con 3. Tener en cuenta que más _chunks_ recuperados y agregados en la prompt implica mayor uso de memoria en inferencia.
* El parámetro `use_chat_template` permite controlar si se aplica el template de chat de Llama (necesario para Llama 3.1, opcional para Gemini). Para Llama usar `True`, para Gemini se puede probar con `True` o `False` según el formato que se prefiera.

In [ ]:
def create_prompt(question, use_chat_template=True, model_for_template=None):
    """
    Crea el prompt para el modelo de lenguaje incluyendo contexto recuperado.
    
    Args:
        question: La pregunta del usuario
        use_chat_template: Si True, aplica el template de chat de Llama
        model_for_template: Modelo del cual usar el tokenizer (solo para Llama)
        
    Returns:
        El prompt formateado
    """
    # Su código aquí
    # 1. Recuperar chunks relevantes usando búsqueda semántica (agregar prefijo "query: " a la pregunta)
    # 2. Construir el contexto con los chunks recuperados
    # 3. Crear el mensaje del sistema y del usuario
    # 4. Aplicar chat template si corresponde
    
    return prompt

Probar la prompt anterior con un ejemplo.

In [ ]:
question = ""  # Completar con una pregunta adecuada al contexto

# Crear prompt según el modelo activo
use_template = isinstance(active_model, LlamaWrapper)  # True para Llama, False para Gemini
prompt = create_prompt(question, use_chat_template=use_template)

print(f"MODELO: {active_model.get_model_name()}")
print("\nPROMPT:")
print(prompt)
print("\nRESPUESTA:")
print(get_response(prompt))

## Parte 5: Evaluación
A continuación vamos a evaluar la solución construida. Para ello, se deben seguir los siguientes pasos:

* Construir un conjunto de evaluación de forma manual que contenga al menos 12 preguntas y respuestas con las siguientes características:
  * Al menos 3 preguntas deben necesitar información presente en más de un _chunk_ para ser respondidas correctamente.
  * Al menos 3 preguntas no deben estar relacionadas con el dominio, y su respuesta de referencia debe ser algo similar a: "Lo siento, no cuento con información para responder esa pregunta."
* El conjunto debe estar en un archivo CSV llamado testset.csv, con las columnas "question" y "answer".

Se deberá realizar al menos tres experimentos diferentes y evaluar sobre el mismo conjunto de test con la métrica BERTScore. Los experimentos deben variar en al menos uno de los siguientes elementos:
* Método de chunking
* Modelo (o método) de retrieval
* Modelo de generación (LLM)
* Método de prompting (se puede probar con few-shot, chain of thought, etc)
* Otros aspectos que considere relevantes a probar

A continuación se definen funciones auxiliares para la evaluación.


In [ ]:
import evaluate
import numpy as np
from tqdm.notebook import tqdm

def generate_predictions(questions, model=None, use_chat_template=True):
    if model is None:
        model = active_model
    
    predictions = []
    for question in tqdm(questions, desc=f"Generando con {model.get_model_name()}"):
        prompt = create_prompt(question, use_chat_template=use_chat_template)
        prediction = get_response(prompt, model=model)
        predictions.append(prediction)
    
    return predictions

def evaluate_predictions(predictions, references, experiment_name=""):
    """Evalúa predicciones usando BERTScore."""
    bertscore = evaluate.load("bertscore")
    results = bertscore.compute(predictions=predictions, references=references, lang='es')
    
    metrics = {
        'precision': np.array(results['precision']).mean(),
        'recall': np.array(results['recall']).mean(),
        'f1': np.array(results['f1']).mean()
    }
    
    if experiment_name:
        print(f"\n=== Resultados: {experiment_name} ===")
    print(f"BERTScore P:  {metrics['precision']:.3f}")
    print(f"BERTScore R:  {metrics['recall']:.3f}")
    print(f"BERTScore F1: {metrics['f1']:.3f}")
    
    return metrics

In [ ]:
import pandas as pd

# Leer el conjunto de evaluación
df = pd.read_csv("testset.csv")

# Obtener preguntas y respuestas
questions = df["question"].tolist()
references = df["answer"].tolist()

Evalúe los experimentos realizados.

In [ ]:
# Evaluar experimentos
# Almacenar resultados para comparación posterior
results_dict = {}

# ============================================================================
# EXPERIMENTO 1: Llama 3.1 con chat template
# ============================================================================
exp1_model = llama_model
exp1_name = "Exp1: Llama 3.1 con chat template"

predictions_exp1 = generate_predictions(questions, model=exp1_model, use_chat_template=True)
results_dict[exp1_name] = evaluate_predictions(predictions_exp1, references, exp1_name)


# ============================================================================
# EXPERIMENTO 2: Gemini 2.0 Flash
# ============================================================================
# exp2_model = gemini_model
# exp2_name = "Exp2: Gemini 2.0 Flash"
# 
# predictions_exp2 = generate_predictions(questions, model=exp2_model, use_chat_template=False)
# results_dict[exp2_name] = evaluate_predictions(predictions_exp2, references, exp2_name)


# ============================================================================
# EXPERIMENTO 3: Otro experimento (acá variamos lo que dijimos)
# ============================================================================
# exp3_model = active_model  # Usar el modelo que ande mejor, por ej.
# exp3_name = "Exp3: [Descripción del experimento]"
# 
# # Ejemplo: se puede modificar create_prompt para usar diferente estrategia
# predictions_exp3 = generate_predictions(questions, model=exp3_model, use_chat_template=True)
# results_dict[exp3_name] = evaluate_predictions(predictions_exp3, references, exp3_name)


# ============================================================================
# RESUMEN DE RESULTADOS
# ============================================================================
print("\n" + "="*80)
print("RESUMEN DE TODOS LOS EXPERIMENTOS")
print("="*80)

import pandas as pd
summary_df = pd.DataFrame(results_dict).T
summary_df.columns = ['Precision', 'Recall', 'F1']
print(summary_df.to_string())

### Reporte de resultados

Reportar los resultados obtenidos en los experimentos realizados completando la siguiente tabla:

| Exp | Descripción | P BERTScore | R BERTScore | F BERTScore |
|-----|-------------|-------------|-------------|-------------|
| 1 | | | | |
| 2 | | | | |
| 3 | | | | |

Responda las siguientes preguntas:

1. Explique brevemente las diferencias en los experimentos realizados, ¿Qué aspectos se varió en el pipeline de RAG?

2. ¿Son consistentes los resultados obtenidos con lo que esperaba?

3. ¿Le parece que la métrica BERTScore está capturando correctamente las diferencias de los distintos experimentos realizados?

(sus respuestas aquí)
